# Example notebook to use the ee-water-extraction tool

First, you need to create a Google Cloud Project with the EarthEngine API enabled. 
Then create a service account with the EarthEngine Viewer role, and download a login json key for this service account. 

## Login to Google Cloud

The next cell load Earth Engine package and login to the API using the service account.

In [ ]:
import ee

credentials = ee.ServiceAccountCredentials('ee-water-extraction@earthengine-371715.iam.gserviceaccount.com', 
                                           './earthengine-key.json')
ee.Initialize(credentials)

## Classical usage workflow
### Import the required packages

In [ ]:
import geemap

from eewaterextraction import data_management
from eewaterextraction import classification
from eewaterextraction import visualization
from eewaterextraction import dgo_metrics

# The next lines are useful if you make modifications in the module files and want to reload it without restarting the notebook kernel
import importlib
importlib.reload(data_management)
importlib.reload(classification)
importlib.reload(visualization)
importlib.reload(dgo_metrics)

### Upload the DGOs to GEE
#### Option 1 
If you have less than 100 DGOs, you can directly upload them with `geemap`.

In [ ]:
# Set the local DGO file path
dgo_shapefile_path = './example_data/Lhasa_RC_DGO2km_updated_gcs.shp'

# Upload it to GEE
dgo_shp = geemap.shp_to_ee(dgo_shapefile_path)

#### Option 2
If you have more than 100 DGOs, you will have to upload them as an asset (in https://code.earthengine.google.com/ for example). Then load it with the next cell.

In [ ]:
dgo_shp = ee.FeatureCollection('projects/earthengine-371715/assets/Yamuna_segm_2km_EPSG_900913')

### Classify images

In [ ]:
# Create a region of interset (union of all the DGOs)
roi = dgo_shp.union(1)

# Get the landsat image collection for your ROI
collection = data_management.getLandsatCollection(start=ee.Date('1980-01-01'), 
                                                  end=ee.Date('2030-12-31'), 
                                                  cloud_filter=80, # Maximum cloud coverage accepted (%)
                                                  cloud_masking=True, # Set to False if you don't want to mask the clouds on accepted images
                                                  roi=roi)

# Calculate MNDWI, NDVI and NDWI
collection = classification.calculateIndicators(collection)

# Classify the objects using the indicators
collection = classification.classifyObjects(collection)

### Layers visualisation

At this point of the workflow, you can create an interactive map for an individual landsat image to check all the previously calculated layers.
This feature is only available in a jupyter notebook.

If you have more than 100 DGOs, this will not be functional (work in progress).

In [ ]:
from ipywidgets import interact, fixed

# Get the landsat_product_id of the collection
landsat_ids = collection.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

map = interact(visualization.imageVisualization, 
         collection=fixed(collection), 
         dgo_shp=fixed(dgo_shp), 
         landsat_id=landsat_ids)

# Show the map in the notebook output
map

### Download layers

If your ROI is not too big, it's possible to download the layers to your local disk. See below the content of each output band.

| band number | content |   
|---|---|
| 1 | blue |
| 2 | green |
| 3 | red |
| 4 | nir |
| 5 | swir1 |
| 6 | swir2 |
| 7 | qa_pixel |
| 8 | mndwi |
| 9 | ndvi |
| 10 | ndwi |
| 11 | water |
| 12 | vegetation |
| 13 | active channel |


In [ ]:
data_management.imageDownload(collection=collection, 
                              landsat_id=landsat_id, 
                              roi=roi, 
                              scale=90, # Downgrading is recommended to reduce the file size
                              output='./example_data/landsat_export.tif')

### Calculate metrics
Execute one of the cell below to select DGOs

In [ ]:
# Select some DGOs
dgo_list = [57,35,42]
selected_dgo = dgo_shp.filter(ee.Filter.inList('DGO_FID', dgo_list))

In [ ]:
# Or select a range of DGOs
dgo_list = list(range(15,50))
selected_dgo = dgo_shp.filter(ee.Filter.inList('DGO_FID', dgo_list))

In [ ]:
# Or select all the DGOs
selected_dgo = dgo_shp

Then calculate the metrics for those DGOs

In [ ]:
# Metrics calculation
metrics = dgo_metrics.calculateDGOsMetrics(collection=collection,
                                           dgos=selected_dgo)

#### Option 1 - Direct download

You can download calculated metrics as pandas dataframe. Technically, the metric calculation is made at this point of the workflow, so the computation time can be long.

In [ ]:
from itertools import chain
import pandas as pd

# Get metrics as pandas dataframe
local_data = metrics.getInfo()
metrics_df = pd.DataFrame([dgo['properties'] for dgo in local_data['features']])

# Save a csv file
metrics_df.to_csv('./example_data/properties.csv')

#### Option 2 - Export to Google Drive

If you have too much DGOs and/or dates, the request will take too much memory and you may need to do a Google Drive export.

In [ ]:
# Set the output Google Drive directory name (your GEE Service Account needs to have write access to this directory)
output_dir = 'GEE_output_metrics'

# Set the output filename
output_name = 'yamuna_metrics'

Start the computation and export task with the cell below.

In [ ]:
import time

task = ee.batch.Export.table.toDrive(  
    collection=metrics,
    folder=output_dir,
    description=output_name,
    selectors=[
        'AC_AREA',
        'CLOUD_SCORE',
        'COVERAGE_SCORE',
        'DATE_ACQUIRED',
        'DGO_FID',
        'LANDSAT_PRODUCT_ID',
        'MEAN_AC_MNDWI',
        'MEAN_AC_NDVI',
        'MEAN_MNDWI',
        'MEAN_NDVI',
        'MEAN_VEGETATION_MNDWI',
        'MEAN_VEGETATION_NDVI',
        'MEAN_WATER_MNDWI',
        'VEGETATION_AREA',
        'VEGETATION_PERIMETER',
        'WATER_AREA',
        'WATER_PERIMETER'
        ],
    fileFormat='CSV')
task.start()

You can check the task status with the following loop.

In [ ]:
while task.active():
    print('\r{}'.format(task.status()), end=" ")
    time.sleep(5)
print('\r{}'.format(task.status()), end=" ")

Download the file from your Google Drive to your local drive and load it in this notebook with the cell below.

#### Option 3 - EarthEngine Asset

In [ ]:
task = ee.batch.Export.table.toAsset(
    collection=metrics,
    description='Lhassa metrics',
    assetId='projects/earthengine-371715/assets/lhassa_metrics'
)
task.start()

In [ ]:
while task.active():
    print('\r{}'.format(task.status()), end=" ")
    time.sleep(5)
print('\r{}'.format(task.status()), end=" ")

## Metrics

| metric name | description |   
|---|---|
| AC_AREA | Active Channel area (pixels) |
| CLOUD_SCORE | Percent of the DGO covered by clouds (%) |
| COVERAGE_SCORE | Percent of the DGO covered by the Landsat image (%) |
| MEAN_AC_MNDWI | Mean MNDWI in the active channel surface |
| MEAN_AC_NDVI | Mean NDVI in the active channel surface |
| MEAN_DRY_MNDWI | Mean MNDWI in the surface which is not water  |
| MEAN_MNDWI | Mean MNDWI of the full DGO |
| MEAN_NDVI| Mean NDVI of the full DGO |
| MEAN_VEGETATION_MNDWI | Mean MNDWI in the vegetation surface |
| MEAN_VEGETATION_NDVI | Mean NDVI in the vegetation surface |
| MEAN_WATER_MNDWI | Mean MNDWI in the water surface |
| VEGETATION_AREA | Vegetation area (pixels) |
| VEGETATION_PERIMETER | Vegetation surface perimeter (projection unit) |
| WATER_AREA | Water area (pixels) |
| WATER_PERIMETER | Water surface perimeter (projection unit) |